In [1]:
#! pip install gensim
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim.test.utils import get_tmpfile

In [3]:
df = pd.read_csv('flaskexample/data/sephora_review_db.csv')
df.head()

,brand,name,brand_id,brand_image_url,product_id,product_image_url,rating,incentivized_review,age,beauty_insider,user_name,review_text,price,recommended,first_submission_date,last_submission_date,location,description
0,Yves Saint Laurent,Mini Black Opium & Mon Paris Eau de Parfum Duo,1070,/contentimages/brands/yvessaintlaurent/1070_lo...,P419415,https://www.sephora.com/productimages/sku/s194...,4,False,NaN,NaN,Zikicheeks,"Love the scents, not a fan of the packaging. I...",$29.00,True,2019-05-15T17:16:53.000+00:00,2019-05-31T06:32:58.000+00:00,NaN,Yves Saint Laurent introduces a new sparkling ...
1,Yves Saint Laurent,Mini Black Opium & Mon Paris Eau de Parfum Duo,1070,/contentimages/brands/yvessaintlaurent/1070_lo...,P419415,https://www.sephora.com/productimages/sku/s194...,5,NaN,18to24,NaN,abbiebee,I got this around New years and I have been us...,$29.00,True,2018-01-24T05:53:51.000+00:00,2018-01-24T06:00:02.000+00:00,NaN,Yves Saint Laurent introduces a new sparkling ...
2,Yves Saint Laurent,Mini Black Opium & Mon Paris Eau de Parfum Duo,1070,/contentimages/brands/yvessaintlaurent/1070_lo...,P419415,https://www.sephora.com/productimages/sku/s194...,4,NaN,NaN,NaN,SydneyMarieHuff,This is really quite nice the only problem is ...,$29.00,True,2017-09-19T18:20:19.000+00:00,2019-06-09T12:07:53.000+00:00,NaN,Yves Saint Laurent introduces a new sparkling ...
3,Yves Saint Laurent,Mini Black Opium & Mon Paris Eau de Parfum Duo,1070,/contentimages/brands/yvessaintlaurent/1070_lo...,P419415,https://www.sephora.com/productimages/sku/s194...,5,NaN,NaN,NaN,Emme17,ABSOLUTELY WORTH EVERY PENNY!!! I love both so...,$29.00,True,2017-09-06T16:51:14.000+00:00,2019-05-05T01:40:26.000+00:00,NaN,Yves Saint Laurent introduces a new sparkling ...
4,Yves Saint Laurent,Mini Black Opium & Mon Paris Eau de Parfum Duo,1070,/contentimages/brands/yvessaintlaurent/1070_lo...,P419415,https://www.sephora.com/productimages/sku/s194...,5,NaN,NaN,NaN,JNMburk,I have both of these and all I can say is OMG!...,$29.00,True,2017-11-18T14:13:42.000+00:00,2019-04-23T05:11:58.000+00:00,NaN,Yves Saint Laurent introduces a new sparkling ...


In [4]:
df.shape

(88004, 18)

In [5]:
#clean the dataset, drop some useless products
drop_product = ['’REPLICA’ Beach Walk Travel Spray','’REPLICA’ By The Fireplace Travel Spray', '’REPLICA’ Jazz Club Travel Spray',
               '’REPLICA’ Lazy Sunday Morning Travel Spray', '’REPLICA’ Sailing Day Travel Spray','Alien Travel Spray',
                'Amazing Grace Firming Body Emulsion','Amazing Grace Shampoo, Bath & Shower Gel','Clémentine California Cologne Pure Perfume Travel Spray',
                'Crystal Mini Trio','Deluxe Perfume Sampler','Eau De Soleil Blanc Travel Spray 10mL','Eau Parfumée Au Thé Blanc Body Lotion',
                'English Pear & Freesia Scent Surround™ Diffuser','Fragrance Combining Travel Duo','Fucking Fabulous Candle',
                'Garden Collection Coffret Set','Grapefruit Reed Diffuser','Lime Basil & Mandarin Scent Surround™ Diffuser',
                "Men's_Cologne_Sampler", 'Mon Paris Eau de Parfum Makeup Gift Set','No.04 Bois de Balincourt Candle',
                'Not A Perfume Travel Spray','Peace Blend Box','Perfume Travel Sampler',"PLAY! by Sephora: Cheers to Beauty",
                "PLAY! by Sephora: Lovestruck Beauty","PLAY! by Sephora: Lovestruck Beauty","PLAY! by Sephora: Scary Good Beauty: Foundations","PLAY! by Sephora: Scary-Good Beauty",
                "PLAY! by Sephora: Your Beauty Future",'Red Roses Scent Surround™ Diffuser','REPLICA Travel Spray Set (Beach Walk, Lazy Sunday Morning, Jazz Club)',
                'REPLICA Under the Lemon Trees Eau de Toilette Travel Spray','Soleil Blanc Shimmering Body Oil','Stone Diffuser',
                'Tobacco Vanille Atomizer','Vanilla Vibes Travel Spray','World of Acqua Di Gio Gift Set'
                ]


In [6]:
for product in drop_product:
    print("total review of {}:".format(product),df[df.name == product].shape[0])
    df = df[df.name != product]
    

total review of ’REPLICA’ Beach Walk Travel Spray: 21
total review of ’REPLICA’ By The Fireplace Travel Spray: 17
total review of ’REPLICA’ Jazz Club Travel Spray: 151
total review of ’REPLICA’ Lazy Sunday Morning Travel Spray: 4
total review of ’REPLICA’ Sailing Day Travel Spray: 11
total review of Alien Travel Spray: 29
total review of Amazing Grace Firming Body Emulsion: 1323
total review of Amazing Grace Shampoo, Bath & Shower Gel: 1383
total review of Clémentine California Cologne Pure Perfume Travel Spray: 123
total review of Crystal Mini Trio: 14
total review of Deluxe Perfume Sampler: 32
total review of Eau De Soleil Blanc Travel Spray 10mL: 1
total review of Eau Parfumée Au Thé Blanc Body Lotion: 15
total review of English Pear & Freesia Scent Surround™ Diffuser: 16
total review of Fragrance Combining Travel Duo: 10
total review of Fucking Fabulous Candle: 3
total review of Garden Collection Coffret Set: 39
total review of Grapefruit Reed Diffuser: 13
total review of Lime Basi

In [7]:
df.shape

(84078, 18)

In [8]:
#total number of products after selection
df.groupby('name')['name'].nunique().shape

(255,)

In [10]:
# combine all reviews to a single column
df_review = pd.DataFrame()
df_review['product_id'] = df.groupby('name').first()['product_id']
df_review['rating'] = df.groupby('name')['rating'].mean()
df_review['description'] = df.groupby('name').first()['description']
df_review['product_image_url'] = df.groupby('name').first()['product_image_url']
df_review['review_text_combine'] = df.groupby('name')['review_text'].apply(lambda x: " ".join(str(v) for v in x))
df_review['price'] = df.groupby('name').first()['price']
df_review=df_review.round({'rating': 1})
df_review.reset_index(inplace = True)
df_review.head(5)

,name,product_id,rating,description,product_image_url,review_text_combine,price
0,1 Million,P269120,4.7,This high-impact fragrance creates a multiface...,https://www.sephora.com/productimages/sku/s120...,My favorite men’s fragrance. My husband was we...,$70.00 - $118.00
1,1 Million Lucky,P431565,4.8,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s203...,"Everytime I spray this on in the morning, I'm ...",$70.00 - $90.00
2,212,P4437,4.6,"A long-lasting, off beat, light floral fragran...",https://www.sephora.com/productimages/sku/s110...,its one of my favorites....I always get compli...,$84.00 - $106.00
3,3 L’IMPERATRICE,P245901,4.6,Succulent exotic fruits and bright pink floral...,https://www.sephora.com/productimages/sku/s120...,I love this scent. I found it a few years ago ...,$80.00
4,ALLURE HOMME SPORT EAU EXTRÊME Eau de Parfum,P377719,5.0,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s141...,This is one of my favorites (besides Polo Red)...,$95.00 - $120.00


In [65]:
#split the description part which contains the "Fragrance Family","Type",and "Key Notes" information
description = df_review['description']
description_split = description.str.extract(r'(Fragrance Family:)(.*)(Type:)(.*)(Key Notes:)(.*)((About:)|(Description:))(.*)')
description_split.head(10)

df_review['Fragrance Family'] = description_split[1]
df_review['Type'] = description_split[3]
df_review['Key Notes'] = description_split[5]
df_review['About'] = description_split[9]

display_list = description_split[9].notna()
df_review['display']=df_review['description']
df_review['display'].loc[display_list] = df_review['About']



In [66]:
#repeat for price
price = df_review['price']
price_split = price.str.extract(r'(.*)-(.*)')
price_split.head(10)

price_list = price_split[0].notna()
df_review['price'].loc[price_list] = price_split[0].loc[price_list]

In [68]:
df_review.head(10)

,name,product_id,rating,description,product_image_url,review_text_combine,price,Fragrance Family,Type,Key Notes,About,text_tot,display
0,1 Million,P269120,4.7,This high-impact fragrance creates a multiface...,https://www.sephora.com/productimages/sku/s120...,My favorite men’s fragrance. My husband was we...,$70.00,NaN,NaN,NaN,NaN,My favorite men’s fragrance. My husband was ...,This high-impact fragrance creates a multiface...
1,1 Million Lucky,P431565,4.8,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s203...,"Everytime I spray this on in the morning, I'm ...",$70.00,Earthy & WoodyScent,Warm Woods,"Hazelnut, Green Plum, Cedar","With unlimited potential and endless fun, lif...","Hazelnut, Green Plum, Cedar With unlimited p...","With unlimited potential and endless fun, lif..."
2,212,P4437,4.6,"A long-lasting, off beat, light floral fragran...",https://www.sephora.com/productimages/sku/s110...,its one of my favorites....I always get compli...,$84.00,NaN,NaN,NaN,NaN,its one of my favorites....I always get comp...,"A long-lasting, off beat, light floral fragran..."
3,3 L’IMPERATRICE,P245901,4.6,Succulent exotic fruits and bright pink floral...,https://www.sephora.com/productimages/sku/s120...,I love this scent. I found it a few years ago ...,$80.00,NaN,NaN,NaN,NaN,I love this scent. I found it a few years ag...,Succulent exotic fruits and bright pink floral...
4,ALLURE HOMME SPORT EAU EXTRÊME Eau de Parfum,P377719,5.0,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s141...,This is one of my favorites (besides Polo Red)...,$95.00,Earthy & WoodyScent,Warm Woods,"Mandarin, Cypress, Tonka Bean","Allure to the extreme. A powerful, dynamic, a...","Mandarin, Cypress, Tonka Bean Allure to the ...","Allure to the extreme. A powerful, dynamic, a..."
5,ALLURE HOMME SPORT Eau de Toilette,P70400,5.0,Fragrance Family: FreshScent Type: Fresh Aquat...,https://www.sephora.com/productimages/sku/s772...,I am extremely picky with my colognes and woul...,$75.00,FreshScent,Fresh Aquatics,"Mandarin, Cedar, White Musk","Allure in motion. More than a fragrance, ALLU...","Mandarin, Cedar, White Musk Allure in motion...","Allure in motion. More than a fragrance, ALLU..."
6,Acqua Di Gio Pour Homme,P12430,4.8,Fragrance Family: FreshScent Type: Fresh Aquat...,https://www.sephora.com/productimages/sku/s397...,i love the smell i keep on hugging my husband ...,$68.00,FreshScent,Fresh Aquatics,"Bergamont, RosemaryFragrance",A resolutely masculine fragrance born from th...,"Bergamont, RosemaryFragrance A resolutely m...",A resolutely masculine fragrance born from th...
7,Acqua Di Gio Profumo,P398665,4.7,Fragrance Family: Earthy & WoodyScrent Type: C...,https://www.sephora.com/productimages/sku/s173...,I bought this cologne for my boyfriend and it ...,$75.00,Earthy & WoodyScrent,Citrus & Woods,"Incense, Sea SaltFragrance","Woody, aromatic, and aquatic, Acqua Di Giò Pr...","Incense, Sea SaltFragrance Woody, aromatic,...","Woody, aromatic, and aquatic, Acqua Di Giò Pr..."
8,Acqua di Gio Absolu,P430348,4.6,Fragrance Family: Earthy & Woody Scent Type: C...,https://www.sephora.com/productimages/sku/s205...,Bought this cologne for my husband as a Father...,$75.00,Earthy & Woody Scent,Citrus & Woods,"Grapefruit, Calone, Labdanum","With Acqua Di Giò Absolu, Giorgio Armani cont...","Grapefruit, Calone, Labdanum With Acqua Di ...","With Acqua Di Giò Absolu, Giorgio Armani cont..."
9,Alien,P196516,4.6,Fragrance Family: FloralScent Type: Warm Flora...,https://www.sephora.com/productimages/sku/s104...,"Next to Tom Ford Noir Pour Femme, this is hand...",$30.00,FloralScent,Warm Floral,"Jasmine Sambac, Cashmeran Wood, Amber Gris",Alien is a floral-woody-amber fragrance that ...,"Jasmine Sambac, Cashmeran Wood, Amber Gris A...",Alien is a floral-woody-amber fragrance that ...


In [69]:
df_review['text_tot'] = df_review['Key Notes'].fillna('') + ' ' + df_review['About'].fillna('') + ' ' + df_review['review_text_combine'] 
df_review.head()

,name,product_id,rating,description,product_image_url,review_text_combine,price,Fragrance Family,Type,Key Notes,About,text_tot,display
0,1 Million,P269120,4.7,This high-impact fragrance creates a multiface...,https://www.sephora.com/productimages/sku/s120...,My favorite men’s fragrance. My husband was we...,$70.00,NaN,NaN,NaN,NaN,My favorite men’s fragrance. My husband was ...,This high-impact fragrance creates a multiface...
1,1 Million Lucky,P431565,4.8,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s203...,"Everytime I spray this on in the morning, I'm ...",$70.00,Earthy & WoodyScent,Warm Woods,"Hazelnut, Green Plum, Cedar","With unlimited potential and endless fun, lif...","Hazelnut, Green Plum, Cedar With unlimited p...","With unlimited potential and endless fun, lif..."
2,212,P4437,4.6,"A long-lasting, off beat, light floral fragran...",https://www.sephora.com/productimages/sku/s110...,its one of my favorites....I always get compli...,$84.00,NaN,NaN,NaN,NaN,its one of my favorites....I always get comp...,"A long-lasting, off beat, light floral fragran..."
3,3 L’IMPERATRICE,P245901,4.6,Succulent exotic fruits and bright pink floral...,https://www.sephora.com/productimages/sku/s120...,I love this scent. I found it a few years ago ...,$80.00,NaN,NaN,NaN,NaN,I love this scent. I found it a few years ag...,Succulent exotic fruits and bright pink floral...
4,ALLURE HOMME SPORT EAU EXTRÊME Eau de Parfum,P377719,5.0,Fragrance Family: Earthy & WoodyScent Type: Wa...,https://www.sephora.com/productimages/sku/s141...,This is one of my favorites (besides Polo Red)...,$95.00,Earthy & WoodyScent,Warm Woods,"Mandarin, Cypress, Tonka Bean","Allure to the extreme. A powerful, dynamic, a...","Mandarin, Cypress, Tonka Bean Allure to the ...","Allure to the extreme. A powerful, dynamic, a..."


In [70]:
#Fragrance Family
df_review['Fragrance Family'].unique()

array([nan, ' Earthy & WoodyScent ', ' FreshScent ',
       ' Earthy & WoodyScrent ', ' Earthy & Woody Scent ',
       ' FloralScent ', ' Warm & SpicyScent ', ' Floral Scent ',
       'Warm & SpicyScent ', ' Warm & Spicy Scent ',
       ' Warm and SpicyScent ', 'FloralScent ', ' FloralsScent ',
       ' Woody & EarthyScent ', ' Fresh Scent ', 'FreshScent '],
      dtype=object)

In [72]:
## only total of four famalies in addition to NaN
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace('Earthy & WoodyScent ', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace('Earthy & WoodyScent', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(' Earthy & Woody Scent ', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace('Earthy & Woody Scent ', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(' Earthy & WoodyScrent ', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(' Earthy & Woody Scent', 'Earthy & Woody Scent')

df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace('Woody & EarthyScent ', 'Earthy & Woody Scent')
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(' FloralScent ', "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("FloralScent ", "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" FloralsScent ", "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("FloralsScent ", "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" Floral Scent ", "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("Floral Scent ", "Floral Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" FreshScent ", "Fresh Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" Fresh Scent ", "Fresh Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("FreshScent ", "Fresh Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("Fresh Scent ", "Fresh Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" Warm & SpicyScent ", "Warm & Spicy Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" Warm & Spicy Scent ", "Warm & Spicy Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("Warm & SpicyScent ", "Warm & Spicy Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace(" Warm and SpicyScent ", "Warm & Spicy Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("Warm & Spicy Scent ", "Warm & Spicy Scent")
df_review['Fragrance Family'] = df_review['Fragrance Family'].str.replace("Warm and SpicyScent ", "Warm & Spicy Scent")

df_review['Fragrance Family'].unique()



array([nan, 'Earthy & Woody Scent', 'Fresh Scent', 'Floral Scent',
       'Warm & Spicy Scent'], dtype=object)

In [74]:
#Type
df_review['Type'] = df_review['Type'].str.replace(' Warm Woods', 'Warm Woods')
df_review['Type'] = df_review['Type'].str.replace(' Warm & Sweet Gourmands', 'Warm & Sweet Gourmands')
df_review['Type'] = df_review['Type'].str.replace(' Warm and Sweet Gourmands', 'Warm & Sweet Gourmands')
df_review['Type'] = df_review['Type'].str.replace('Warm & Sweet Gourmands ', 'Warm & Sweet Gourmands')
df_review['Type'] = df_review['Type'].str.replace('Warm and Sweet Gourmands', 'Warm & Sweet Gourmands')
df_review['Type'] = df_review['Type'].str.replace(' Warm & Sweet Gourmand', 'Warm & Sweet Gourmands')

df_review['Type'] = df_review['Type'].str.replace(' Cool Spices ', 'Cool Spices')
df_review['Type'] = df_review['Type'].str.replace(' Cool Spices', 'Cool Spices')
df_review['Type'] = df_review['Type'].str.replace('Cool Spices ', 'Cool Spices')
df_review['Type'] = df_review['Type'].str.replace(' Warm Floral', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace('Warm Floralss ', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace('Warm Florals ', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace('Warm Floral ', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace('Warm Floral', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace('Warm Floralss', 'Warm Florals')
df_review['Type'] = df_review['Type'].str.replace(' Fresh Citrus & Fruit', 'Fresh Citrus & Fruits')
df_review['Type'] = df_review['Type'].str.replace('Fresh Citrus & Fruit', 'Fresh Citrus & Fruits')
df_review['Type'] = df_review['Type'].str.replace('Fresh Citrus & Fruitss', 'Fresh Citrus & Fruits')
df_review['Type'] = df_review['Type'].str.replace(' Fresh Citrus and Fruits', 'Fresh Citrus & Fruits')

df_review['Type'] = df_review['Type'].str.replace(' Fresh Aquatics', 'Fresh Aquatics')
df_review['Type'] = df_review['Type'].str.replace('Fresh Aquatics ', 'Fresh Aquatics')
df_review['Type'] = df_review['Type'].str.replace(' Citrus & Woods ', 'Citrus & Woods')
df_review['Type'] = df_review['Type'].str.replace(' Citrus & Woods', 'Citrus & Woods')
df_review['Type'] = df_review['Type'].str.replace('Citrus & Woods ', 'Citrus & Woods')
df_review['Type'] = df_review['Type'].str.replace(' Classic Woods', 'Classic Woods')
df_review['Type'] = df_review['Type'].str.replace(' Classic Woods ', 'Classic Woods')
df_review['Type'] = df_review['Type'].str.replace('Classic Woods ', 'Classic Woods')
df_review['Type'] = df_review['Type'].str.replace('Classic Florals ', 'Classic Florals')
df_review['Type'] = df_review['Type'].str.replace('Fruity Floral', 'Fruity Florals')
df_review['Type'] = df_review['Type'].str.replace('Fruity Floralss', 'Fruity Florals')
df_review['Type'] = df_review['Type'].str.replace('Fruity Floralss ', 'Fruity Florals')
df_review['Type'] = df_review['Type'].str.replace('Fruity Floralss', 'Fruity Florals')

df_review['Type'] = df_review['Type'].str.replace('Fresh Floral', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Florals ', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Floralssss', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Floralss ', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Floralsss ', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Floralsss', 'Fresh Florals')
df_review['Type'] = df_review['Type'].str.replace('Fresh Floralss', 'Fresh Florals')

df_review['Type'] = df_review['Type'].str.replace('Woody Spice ','Woody Spices')
df_review['Type'] = df_review['Type'].str.replace('Woody Spices ','Woody Spices')
df_review['Type'].unique()

array([nan, 'Warm Woods', 'Fresh Aquatics', 'Citrus & Woods',
       'Warm Florals', ' Classic Floral', ' Powdery Florals',
       'Warm & Sweet Gourmands', 'Fresh Citrus & Fruits',
       ' Fresh Florals', ' Fruity Florals ', ' Earthy Greens & Herbs',
       ' Woody Spices', 'Warm Floralss', 'Fresh Citrus & Fruitss',
       ' Classic Florals', ' Fresh Fruity Florals', ' Fruity Florals',
       ' Woody Spice', 'Cool Spices', ' Powdery Floral ', 'Classic Woods',
       'Fresh Citrus & Fruitss ', ' Powdery Floral'], dtype=object)

In [ ]:
#df_review.to_csv('flaskexample/data/fragrance_data.csv', index=False)